# Rotation Invariant Quantization (RIQ) 

This notebook provides an easy step-by-step walkthrough for downloading off-the-shelf model, quantizing the model with RIQ  such that a certain cosine distance requirement
is satisfied at the model's output. Then, compressing the quantized model with ANS layer-by-layer. 

You will:

 * Set up the environment
 * Select a model
 * Download the model
 * Quantize the model with RIQ and compress the it using ANS encoder
     * The compressed model is saved into a dictonary file 
 * Load the compressed model from a file into onnx format  
 * Run inference with the quantized weights and compare its output to the original model     
    
 Reading through this notebook will quickly provide an overview of the RIQ approach, and its performance. 

#### Setup the environment

In [ ]:
import torch
import pickle
import numpy as np
from tqdm import tqdm
from torchvision.models import vgg16, VGG16_Weights, resnet50, ResNet50_Weights, alexnet, AlexNet_Weights

from utils.quantize import get_quantized_model, bytes_to_bitstring, measure_cos_sim
from utils.dataset import prepare_dataset_images
from utils.onnx_bridge import OnnxBridge

#### Select a model

* pick a model (resnet, vgg or alexnet)
 

In [ ]:
picked_model = "resnet"
model_path = "models/" + picked_model + ".onnx"

model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1) if picked_model =="resnet" else \
            vgg16(weights=VGG16_Weights.IMAGENET1K_V1) if picked_model =="vgg" else \
                alexnet(weights=AlexNet_Weights.IMAGENET1K_V1) if picked_model =="alexnet" else None

if model != None:
    model.eval()
    inp = torch.randn(1, 3, 224, 224)
    in_names = ["actual_input"]
    out_name = ["output"]
    torch.onnx.export(model, inp, model_path, verbose=False, input_names=in_names, output_names=out_name, export_params=True)
else:
    print("Please pick one of the following: \"resnet\", \"vgg\", \"alexnet\". ")

* set path to a small set of calibtration images
    * This is optional, however, **better compression results are attain with a small calibration set**
    * leave the calibration_path="" for no cailbration data   

In [ ]:
calibration_path = "<path to small set of pictures (optional, you can leave an empty string)>" 
calibration_data = prepare_dataset_images(calibration_path+"*.jpg", model_path)

#### Quantize with RIQ and compress with ANS 

For the given distortion requirement at the output (0.005 in this demonstration):
* RIQ finds a solution with minimal entropy 
* compress it with ANS layer by layer
* save the compressed weights to a file .riq
    * This enables to verify the reported compression ratio 
    * Enables to use these compressed weights in the sequel for inference

In [ ]:
filename = picked_model + '.riq'
get_quantized_model(model_path, calibration_data, distortion=0.005, save_to=filename)

#### Load and Decode compressed model

The weights are loaded into a dict which we use for comparison with the original model

In [ ]:
with open(filename, 'rb') as fn:
    load_compressed_weights_dict = pickle.load(fn)
    
qws = {}
for k, tup in tqdm(load_compressed_weights_dict.items()):    
    tans, bytestream, delta, shape = tup    
    bitstream  = bytes_to_bitstring(bytestream)    
    if delta < 1.0 :        
        qw = tans.decode_data(bitstream)
        qw = np.array(qw).reshape(shape)
        qw = qw * delta 
        qws[k] = qw
    else: 
        qws[k] = bytestream 
    

#### Compare outputs of the model

This part requires small validation set for comparing the output.

* For the calibratoin set can be used for this comparison as well (thought doesn't reflect )

In [ ]:
ob = OnnxBridge(model_path) #load original model
validation_dataset = calibration_data 
original_outs = np.array([ob(i) for i in validation_dataset])
ob.set_weights(qws)
outs = np.array([ob(i) for i in validation_dataset])
cos_distance = 1-measure_cos_sim(original_outs, outs, _)
print("cos distance is ", cos_distance)